# Benchmark


In this notebook, we use pepme to evalulate antimicrobial peptide sequences.


In [ ]:
import numpy as np

from pepme import FeatureCache, compute_metrics, show_table
from pepme.metrics import FBD, ID, MMD, AuthPct, ConformityScore, Fold, Novelty, Precision, Recall, Uniqueness
from pepme.models import Charge, Esm2, Esm2Checkpoint, Gravy
from pepme.utils import random_subset, read_fasta_file, shuffle_sequences

Let's load the datasets.


In [ ]:
DATASET_PATHS = {
    # Databases
    "UniProt": "./data/uniprot/uniprot_8_50_100.fasta",
    "AMP-data": "./data/amps.fasta",
    "AMPs": "./data/amps_hq.fasta",
    "DBAASP": "./data/dbaasp.fasta",
    # Models
    "AMP-Diffusion": "./data/amp-diffusion.fasta",
    "AMP-GAN": "./data/amp-gan.fasta",
    "CPL-Diff": "./data/cpl-diff.fasta",
    "HydrAMP": "./data/hydramp.fasta",
    "OmegAMP": "./data/omegamp.fasta",
}

In [ ]:
datasets = {name: read_fasta_file(path) for name, path in DATASET_PATHS.items()}

In [ ]:
for model_name, sequences in datasets.items():
    print(f"{model_name}: {len(sequences)} sequences")

UniProt: 2933310 sequences
AMP-data: 37116 sequences
AMPs: 7204 sequences
DBAASP: 8967 sequences
AMP-Diffusion: 47671 sequences
AMP-GAN: 150000 sequences
CPL-Diff: 49985 sequences
HydrAMP: 50000 sequences
OmegAMP: 149504 sequences


Let's setup the data and models.


In [ ]:
sources = [
    "AMP-Diffusion",
    "AMP-GAN",
    "CPL-Diff",
    "HydrAMP",
    "OmegAMP",
    "UniProt",
]

n_samples = 40_000  # 3_000
seed = 42

benchmark_datasets = {
    source: random_subset(datasets[source], n_samples=n_samples, seed=seed)
    if len(sequences) > n_samples
    else datasets[source]
    for source in sources
}
# benchmark_datasets["DBAASP (shuffled)"] = shuffle_sequences(datasets["DBAASP"], seed=seed)

seqs_amps = datasets["AMPs"]
seqs_train = datasets["AMP-data"]

In [ ]:
def my_embedder(sequences: list[str]) -> np.ndarray:
    lengths = [len(sequence) for sequence in sequences]
    counts = [sequence.count("K") for sequence in sequences]
    return np.array([lengths, counts]).T


esm2 = Esm2(
    model_name=Esm2Checkpoint.t6_8M,
    batch_size=256,
    device="cpu",
    verbose=False,
)

cache = FeatureCache(
    models={
        "embedder": my_embedder,
        "esm2-embed": esm2.embed,
        "esm2-perplexity": lambda seqs: esm2.compute_pseudo_perplexity(seqs, mask_size=3),
        "gravy": Gravy(),
        "charge": Charge(),
    }
)

Let's select the metrics.


In [ ]:
embedder = "embedder"  # "embedder"


metrics = [
    Uniqueness(),
    Novelty(reference=seqs_train),
    FBD(reference=seqs_amps, embedder=cache.model(embedder)),
    MMD(reference=seqs_amps, embedder=cache.model(embedder)),
    Fold(
        Precision(neighborhood_size=3, reference=seqs_train, embedder=cache.model(embedder), strict=True),
        split_size=len(seqs_train),
        drop_last=True,
    ),
    Fold(
        Recall(neighborhood_size=3, reference=seqs_train, embedder=cache.model(embedder), strict=True),
        split_size=len(seqs_train),
        drop_last=True,
    ),
    # ID(predictor=cache.model("esm2-perplexity"), name="Perplexity", objective="minimize"),
    # @TODO: ID amplify
    # @TODO: ID amPEPpy
    # @TODO: ID y_mic
    Fold(
        AuthPct(train_set=seqs_train, embedder=cache.model(embedder), strict=True),
        split_size=len(seqs_train),
        drop_last=True,
    ),
    # @TODO: FLD (seqs_train)
    ConformityScore(reference=seqs_amps, descriptors=[cache.model("gravy"), cache.model("charge")]),
]

`Fold` computes the metrics multiple using different folds, and aggregate the values (mean and standard deviation).

Wrapping `Fold` around the `Precision` and `Recall` metric, removes the sample size bias inherent in these metrics (introduced by k-NN), while still utilizing as many of the available sequences as possible.


In [ ]:
df = compute_metrics(benchmark_datasets, metrics)

100%|██████████| 48/48 [05:00<00:00,  6.26s/it, data=UniProt, metric=Conformity score]     


Let's look at the results.


In [ ]:
show_table(df, decimals=[2, 2, 2, 2, 2, 2, 2, 2])

,Uniqueness↑,Novelty↑,FBD↓,MMD↓,Precision↑,Recall↑,Authenticity↑,Conformity score↑
AMP-Diffusion,0.91,1.00,56.24,199.98,1.00,0.62,0.01,0.32±0.00
AMP-GAN,1.00,1.00,3.16,45.11,0.95,0.86,0.05,0.43±0.00
CPL-Diff,0.98,0.99,8.04,13.12,0.92,1.00,0.08,0.37±0.00
HydrAMP,1.00,1.00,16.70,37.58,0.99,0.77,0.01,0.26±0.00
OmegAMP,0.95,0.91,20.21,26.00,0.87,1.00,0.13,0.33±0.00
UniProt,1.00,1.00,358.93,741.10,0.99,0.99,0.02,0.22±0.00
